In [3]:
import pandas as pd
import json
import psycopg2

In [2]:
newdf = pd.read_csv('water_cleaned.csv', sep=',')

In [4]:
def create_connection():
    try:
        with open('db_config.json') as file:
            config = json.load(file)
        connection = psycopg2.connect(
            host='localhost',
            user=config["user"],
            password=config["password"],
            database='db_water'
        )
        print('Conexión exitosa!!')
    except psycopg2.Error as e:
        connection = None
        print('No se puede conectar:', e)
    return connection
create_connection()

Conexión exitosa!!


<connection object at 0x0000021864BA2460; dsn: 'user=postgres password=xxx dbname=db_water host=localhost', closed: 0>

Creating a new table to insert the data the new dataframe after EDA

In [6]:
def create_tabla():
    connection = create_connection()
    if connection is not None:
        try:
            cursor = connection.cursor()
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS tabla_data_cleaned (
                        id SERIAL PRIMARY KEY,
                        ano DATE NOT NULL,
                        nombre_departamento VARCHAR(255) NOT NULL,
                        div_dpto INT NOT NULL,
                        nombre_municipio VARCHAR(255) NOT NULL,
                        divi_muni INT NOT NULL,
                        irca_minimo FLOAT NOT NULL,
                        irca_maximo FLOAT NOT NULL,
                        irca_promedio FLOAT NOT NULL,
                        nombre_parametro_analisis2 VARCHAR(255) NOT NULL,
                        numero_parametros_promedio INT NOT NULL,
                        rango_irca VARCHAR(255) NOT NULL,
                        porcentaje_muestras_tratadas FLOAT NOT NULL,
                        diferencia_muestras_tratadas_sin_tratar INT NOT NULL,
                        rango_parametros_analizados INT NOT NULL
                    );
                """)
            connection.commit()
            print("Tabla creada con éxito en la base de datos db_water.")
        except psycopg2.Error as e:
            print(f"Error al crear la tabla en db_water: {e}")
        finally:
            cursor.close()
            connection.close()
    else:
        print("No se pudo establecer la conexión con la base de datos db_water.")

create_tabla()


Conexión exitosa!!
Tabla creada con éxito en la base de datos db_water.


Insert data the resulting dataframe 

In [8]:
def insertar_datos_agua(df):
    connection = create_connection()
    cursor = connection.cursor()
    query = """
    INSERT INTO tabla_data_cleaned (ano, nombre_departamento, div_dpto, nombre_municipio, divi_muni, irca_minimo, irca_maximo, irca_promedio, nombre_parametro_analisis2, numero_parametros_promedio, rango_irca, porcentaje_muestras_tratadas, diferencia_muestras_tratadas_sin_tratar, rango_parametros_analizados)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    try:
        for index, row in df.iterrows():
            data = (row["año"], row["nombre_departamento"], row["div_dpto"], row["nombre_municipio"], row["divi_muni"],
                    row["irca_minimo"], row["irca_maximo"], row["irca_promedio"], row["nombre_parametro_analisis2"],
                    row["numero_parametros_promedio"], row["rango_irca"], row["porcentaje_muestras_tratadas"],
                    row["diferencia_muestras_tratadas_sin_tratar"], row["rango_parametros_analizados"])
            cursor.execute(query, data)
        connection.commit()
        print("Datos insertados exitosamente")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        cursor.close()
        connection.close()
        
insertar_datos_agua(newdf)


Conexión exitosa!!
Datos insertados exitosamente
